[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-advanced-dl/blob/main/Unidad%203%20-%20Transformers/ollama-rag.ipynb)
### References
- https://github.com/LozanoAlvarezb/MLOPS-Newtral/tree/main

In [1]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

In [2]:
!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-advanced-dl/raw/main/Unidad%203%20-%20Transformers/requirements.txt && pip install -r requirements.txt

In [3]:
from datasets import load_dataset

dataset = load_dataset('somosnlp/wikihow_es', split='train')
dataset

Dataset({
    features: ['title', 'section_name', 'summary', 'document', 'english_section_name', 'english_url', 'url'],
    num_rows: 90528
})

In [4]:
import pandas as pd

pd.set_option('display.max_colwidth', 100)
dataset.set_format(type='pandas')
df = dataset.to_pandas()
df.head(15)


,title,section_name,summary,document,english_section_name,english_url,url
0,¿Cómo cultivar Delphinium?,Tomar esquejes de delphinium,"Cuando llegue marzo o abril, escoge brotes nuevos cerca de la base de la planta. Haz el corte ce...","Estos brotes serán jóvenes y sólidos, lo que es importante para un esqueje sano. A medida que la...",Taking Delphinium Cuttings,https://www.wikihow.com/Grow-Delphinium,https://es.wikihow.com/cultivar-delphinium
1,¿Cómo elegir Un buen nombre de dominio para tu sitio web?,Pensar en el futuro,Comprométete con el nombre de dominio. Escoge un nombre que deje espacio para el crecimiento. Ev...,"Cualquiera sea el nombre de dominio que escojas, debes asegurarte de comprometerte con este a la...",Thinking About the Future,https://www.wikihow.com/Pick-a-Good-Domain-Name-for-Your-Website,https://es.wikihow.com/elegir-un-buen-nombre-de-dominio-para-tu-sitio-web
2,¿Cómo hacer Tus propios regalos de navidad?,Convierte fotos en regalos,. Toma una fotografía Prueba las fotos personales.,"ecora un marco de fotos. Compra un marco de fotos barato. Puedes usar plástico, pero otros mater...",Turning Photos Into Gifts,https://www.wikihow.com/Make-Your-Own-Christmas-Gifts,https://es.wikihow.com/hacer-tus-propios-regalos-de-Navidad
3,¿Cómo soñar Despierto?,Saber qué soñar,Sueña con tu futuro deseado. Sueña con tus cosas favoritas. Actúa en tus sueños. Sueña con algo ...,Soñar despierto con una meta en mente puede ayudarte a obtener la motivación para alcanzarla. Im...,Knowing What to Dream About,https://www.wikihow.com/Daydream,https://es.wikihow.com/so%C3%B1ar-despierto
4,¿Cómo planear Una fiesta de navidad que sea elegante?,Comidas y bebidas,Debes decidir si vas a servir bebidas alcohólicas en tu fiesta: Los bocadillos y postres son una...,"en caso de hacerlo, debes tener refrescos o ponche para aquellos que no beban alcohol. También e...",Refreshments and Food,https://www.wikihow.com/Plan-an-Elegant-Christmas-Party,https://es.wikihow.com/planear-una-fiesta-de-navidad-que-sea-elegante
5,¿Cómo ver Las descargas en android?,Usar un administrador de archivos,"Abre la bandeja de aplicaciones. Pulsa Descargas, Mis archivos o Administrador de archivos. Sele...","Esta es la lista de aplicaciones del dispositivo Android. Por lo general, puedes abrirla pulsand...",Using a File Manager,https://www.wikihow.com/View-Downloads-on-Android,https://es.wikihow.com/ver-las-descargas-en-Android
6,¿Cómo deshacerse De los insectos de escama?,Eliminar los insectos de escama del jardín,Usa los mismos métodos que los que usaste para las plantas del interior al combatir un ataque de...,Estos insectos son más predominantes al aire libre donde infestan a todas las especies de árbole...,Removing Scales from the Garden,https://www.wikihow.com/Get-Rid-of-Scale-Insects,https://es.wikihow.com/deshacerse-de-los-insectos-de-escama
7,¿Cómo quitar Una mancha de vino tinto de una mesa de madera?,Mancha fresca de vino tinto,Seca el derrame de vino tinto con un paño absorbente húmedo inmediatamente. Usa una toalla suave...,Asegúrate de sólo secar el área; no lo frotes ya que se podría extender en la superficie de made...,Fresh Red Wine Stain,https://www.wikihow.com/Remove-a-Red-Wine-Stain-Ring-from-a-Wood-Table,https://es.wikihow.com/quitar-una-mancha-de-vino-tinto-de-una-mesa-de-madera
8,¿Cómo adiestrar A un gato con un clicker?,Prepárate para entrenar al gato,Consigue un clicker. Ten algunos premios a la mano. Consigue un artículo objetivo. Encuentra un ...,"Un clicker es una caja de plástico pequeña con una pestaña de metal. Al presionarla, produce un ...",Getting Ready to Train Your Cat,https://www.wikihow.com/Clicker-Train-a-Cat,https://es.wikihow.com/adiestrar-a-un-gato-con-un-clicker
9,¿Cómo hacer Un mate pastor en el ajedrez?,Realizar la jugada,Juega con las piezas blancas. e4: Bc4: Qh5: Qxf7: ¡Jaque Mate!,mueve el peón de tu rey dos cuadros hacia adelante (E2-E4). mueve el alfil del rey 3 cuadros a ...,Performing the Play,https://www.wikihow.com/D

In [5]:
df.shape

(90528, 7)

In [6]:
from typing import Callable, Dict, List, Optional
from collections import defaultdict


def split_text(chunk_size: int, sliding_window: int = 0) -> Callable:
    def _split_text(batch):
        articles = {url: summary.split(' ') + document.split(' ') for url, summary, document in zip(batch['url'], batch['summary'], batch['document'])}
        chunks = defaultdict(list)

        for title, (url, doc_words) in zip(batch['title'], articles.items()):
            buffer = []
            for word in doc_words:
                if len(buffer) < chunk_size:
                    buffer.append(word)
                else:
                    chunk_text = ' '.join(buffer)
                    chunks['title'].append(title)
                    chunks['text'].append(chunk_text)
                    chunks['url'].append(url)
                    if sliding_window == 0:
                        del buffer[:]
                    else:
                        del buffer[0: sliding_window]
                    buffer.append(word)
            if buffer:
                chunk_text = ' '.join(buffer)
                chunks['title'].append(title)
                chunks['text'].append(chunk_text)
                chunks['url'].append(url)

        return chunks
    return _split_text


documents = dataset.map(split_text(chunk_size=100, sliding_window=50), batched=True, remove_columns=dataset.column_names)
documents

Dataset({
    features: ['title', 'url', 'text'],
    num_rows: 734449
})

In [7]:
documents[0:2500:500]

,title,url,text
0,¿Cómo cultivar Delphinium?,https://es.wikihow.com/cultivar-delphinium,"Cuando llegue marzo o abril, escoge brotes nuevos cerca de la base de la planta. Haz el corte ce..."
1,¿Cómo colocar Los espejos según el feng shui?,https://es.wikihow.com/colocar-los-espejos-seg%C3%BAn-el-Feng-Shui,es muy malo para el feng shui de tu hogar. Evita utilizar espejos redondeados u ovalados en tu h...
2,¿Cómo seguir Los requerimientos nutricionales para la anemia?,https://es.wikihow.com/seguir-los-requerimientos-nutricionales-para-la-anemia,están en un riesgo elevado de sufrir de anemia y estos son: las mujeres (a causa de la pérdida ...
3,¿Cómo preparar Vermicompost?,https://es.wikihow.com/preparar-vermicompost,"los granos de café también servirán. Es importante en especial que no agregues carne, huesos ni..."
4,¿Cómo rellenar Tu sostén?,https://es.wikihow.com/colocar-letreros-de-Prohibido-el-Paso,señalización permanezca legible con el tiempo. Tu área puede tener requisitos específicos sobre ...


In [8]:
documents.reset_format()

In [26]:
import os
from pathlib import Path
import numpy as np
from sentence_transformers import SentenceTransformer

model_ckpt = 'intfloat/multilingual-e5-small'
embeddings_path = Path('data/vectorstore.npy')
limit = 5000

model = SentenceTransformer(model_ckpt)

if embeddings_path.exists():
    vectorstore = np.load(str(embeddings_path))
else:
    vectorstore = documents.take(limit).map(lambda example: {'embedding': model.encode(example['text'])}, remove_columns=documents.column_names)
    vectorstore.reset_format()
    vectorstore = np.concatenate([np.array(v['embedding'], dtype=np.float32).reshape(1, -1) for v in vectorstore.to_iterable_dataset()], axis=0)
    os.makedirs(embeddings_path.parent, exist_ok=True)
    np.save(str(embeddings_path), vectorstore)


OutOfMemoryError: CUDA out of memory. Tried to allocate 368.00 MiB. GPU 0 has a total capacity of 7.78 GiB of which 231.56 MiB is free. Including non-PyTorch memory, this process has 1002.00 MiB memory in use. Process 1583794 has 4.87 GiB memory in use. Of the allocated memory 846.20 MiB is allocated by PyTorch, and 29.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
vectorstore.shape

(1000, 384)

In [11]:
documents.reset_format()

In [12]:
from datasets import Dataset
from sentence_transformers import util as st_util

class DocumentRetriever:

    def __init__(self, documents: Dataset, vectorstore: np.ndarray, model: SentenceTransformer) -> None:
        self.documents = documents
        self.vectorstore = vectorstore
        self.model = model

    def search(self, query: str, k: int = 5):
        q = self.model.encode(query)
        scores = st_util.dot_score(q, self.vectorstore)
        scores = scores.squeeze()

        topk = (-scores).argsort()[:k].numpy().tolist()

        return [{**self.documents[i], "score": scores[i].item()} for i in topk]

In [13]:
retriever = DocumentRetriever(documents, vectorstore, model)
retriever.search('Qué es feng shui?')

[{'title': '¿Cómo colocar Los espejos según el feng shui?',
  'url': 'https://es.wikihow.com/colocar-los-espejos-seg%C3%BAn-el-Feng-Shui',
  'text': 'pero evita aquellos que son pequeños o facetados. Lo aconsejable es poder verte de pies a cabeza. Evita escoger espejos con superficies distorsionadas. Esto también aplica a los espejos con superficies anticuadas. Se cree que observar a través de estos espejos hace que las personas se sientan distorsionadas, y esto es muy malo para el feng shui de tu hogar. Evita utilizar espejos redondeados u ovalados en tu hogar. Por lo general, los espejos cuadrados o rectangulares se consideran mejores para crear un buen feng shui. No cuelgues espejos rotos o rajados en tu hogar. Si un espejo que tienes',
  'score': 0.8721307516098022},
 {'title': '¿Cómo colocar Los espejos según el feng shui?',
  'url': 'https://es.wikihow.com/colocar-los-espejos-seg%C3%BAn-el-Feng-Shui',
  'text': 'es muy malo para el feng shui de tu hogar. Evita utilizar espejos re

In [14]:
from abc import abstractmethod
from typing import Any, Generator

class LLM:

    @abstractmethod
    def completion_stream(self, *args: Any, **kwargs: Any) -> Generator:
        raise NotImplementedError
    

    def completion(self, *args, **kwargs):
        return "".join(self.completion_stream(*args, **kwargs))



In [15]:
from typing import Any, Generator
import ollama

class Ollama(LLM):

    def __init__(self, model: str) -> None:
        super().__init__()
        self.model = model

    def completion_stream(self, messages) -> Generator[Any, None, None]:
        stream = ollama.chat(
            model=self.model,
            messages=messages,
            stream=True
        )
        for chunk in stream:
            content = chunk["message"]["content"]
            token = content if content is not None else ""
            yield token

In [16]:
!if ! type ollama > /dev/null; then curl -fsSL https://ollama.com/install.sh | sh; else echo "Ollama ya está instalado."; fi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Ollama ya está instalado.


In [17]:
!ollama pull llama3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [18]:
llm_ollama = Ollama(model="llama3")

In [19]:
document_separator = "\n\n"

question_template = """Utiliza los siguientes fragmentos de contexto para responder la pregunta al final. Si no sabes la respuesta, di que no lo sabes. Responde como en una conversación natural.

{context}

Pregunta: {question}
Respuesta Útil"""

history_remplate = """Dada la siguiente conversación y la pregunta, expresa de otro modo la pregunta para que todo sea una sola pregunta en general

Historial:
{chat_history}
Siguiente pregunta: {question}
Pregunta general:"""

In [20]:
class ChatBot:

    def __init__(self, document_retriever: DocumentRetriever, llm: LLM, topk: int = 5) -> None:
        self.dr = document_retriever
        self.llm = llm
        self.topk = topk
        self.history = []

    def reset(self):
        del self.history[:]

    def follow_up_query(self, question):
        prompt = history_remplate.format(chat_history="\n".join(self.history), question=question)
        query = self.llm.completion(prompt)
        return query
    
    def __call__(self, question: str, history: bool = False):
        if history and len(self.history):
            query = self.follow_up_query(question)
        else:
            query = question

        documents = self.dr.search(query, k = self.topk)

        contexts = [document['text'] for document in documents]
        context = document_separator.join(contexts)
        prompt = question_template.format(context=context, question=query)

        messages = [{'role': 'system', 'content': prompt}]

        answer = self.llm.completion(messages)

        if history:
            self.history.append('\n'.join([f'Pregunta: {prompt}', f'Respuesta: {answer}']))

        citation = {document['title']: document['url'] for document in documents}
        citation = [f"{idx}. {title} - {url}" for idx, (title, url) in enumerate(citation.items(), 1)]
        return '\n'.join([answer, *citation])
        

In [21]:
chatbot = ChatBot(retriever, llm_ollama)

In [22]:
print(chatbot("¿Qué es el feng shui?"))

Según los fragmentos de contexto que me proporcionaste, el feng shui se refiere a la concepción china de la energía y el espacio. En general, se considera que el feng shui es una práctica que busca crear un equilibrio y armonía en el entorno, utilizando elementos como los espejos, la forma y orientación de las habitaciones, entre otros.
1. ¿Cómo colocar Los espejos según el feng shui? - https://es.wikihow.com/colocar-los-espejos-seg%C3%BAn-el-Feng-Shui
2. ¿Cómo comer Fideos? - https://es.wikihow.com/comer-fideos
3. ¿Cómo lograr Tener paz interior? - https://es.wikihow.com/lograr-tener-paz-interior
4. ¿Cómo hacer Que no te importe lo que la gente piense? - https://es.wikihow.com/hacer-que-no-te-importe-lo-que-la-gente-piense


In [23]:
import gradio as gr

with gr.Blocks() as gr_blocks:
    chatbot = gr.Chatbot()
    bot = ChatBot(retriever, llm_ollama)
    msg = gr.Textbox(
        label="Sobre qué quieres conversar?",
        placeholder="Ház tu pregunta aquí y presiona enter."
    )
    clear = gr.Button("Limpiar")

    def respond(question, chat_history):
        bot_message = bot(question)
        chat_history.append((question, bot_message))
        return "", chat_history
    
    def reset_chat():
        bot.reset()
        return ""
    
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(reset_chat, None, chatbot, queue=False)

gr_blocks.launch(inline=False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [24]:
gr_blocks.close()

Closing server running on port: 7860
